### Тема “Генерация текстов (языковое моделирование)”

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import numpy as np
import tensorflow as tf
import os
import warnings
warnings.filterwarnings('ignore')

2022-11-22 09:54:52.906678: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
text = open('evgenyi_onegin.txt', 'rb').read().decode(encoding='utf-8')

In [3]:
print('Length of text: {} characters'.format(len(text)))

Length of text: 286984 characters


In [4]:
print(text[:700])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                        Вниманье дружбы возлюбя,
                        Хотел бы я тебе представить
                        Залог достойнее тебя,
                        Достойнее души прекрасной,
                        Святой исполненной мечты,
                        Поэзии живой и ясной,
                        Высоких дум и простоты;
                        Но так и быть - рукой пристрастной
                        Прими собранье пестрых глав,
                        Полусмешных, полупечальных,
                


In [5]:
vocab = sorted(set(text))

In [6]:
print('{} unique characters'.format(len(vocab)))

131 unique characters


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print(text[:200]), print(text_as_int[:200])

Александр Сергеевич Пушкин

                                Евгений Онегин
                                Роман в стихах

                        Не мысля гордый свет забавить,
                      
[ 71 110 104 109 116  99 112 103 115   1  87 104 115 102 104 104 101 107
 122   1  85 118 123 109 107 112   0   0   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1  76 101 102 104 112 107 108   1  84 112 104 102
 107 112   0   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1  86
 113 111  99 112   1 101   1 116 117 107 120  99 120   0   0   1   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1
   1   1   1  83 104   1 111 126 116 110 130   1 102 113 115 103 126 108
   1 116 101 104 117   1 106  99 100  99 101 107 117 127   7   0   1   1
   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1   1 

(None, None)

In [9]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

2022-11-22 09:54:57.633360: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
for i in char_dataset.take(10):
    print(idx2char[i.numpy()])

А
л
е
к
с
а
н
д
р
 


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [12]:
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '
'      Роман в стихах\n\n                        Не мысля гордый свет забавить,\n                        '
'Вниманье дружбы возлюбя,\n                        Хотел бы я тебе представить\n                        '
'Залог достойнее тебя,\n                        Достойнее души прекрасной,\n                        Свят'
'ой исполненной мечты,\n                        Поэзии живой и ясной,\n                        Высоких д'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [14]:
for input_example, target_example in dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Александр Сергеевич Пушкин\n\n                                Евгений Онегин\n                         '
Target data: 'лександр Сергеевич Пушкин\n\n                                Евгений Онегин\n                          '


In [15]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 131) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           33536     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 131)           134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899
No

In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1) #количество независимых выборок 1

sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([ 61, 116, 127, 120,  11,   4,  47, 128,  19,  83,  75,  53, 105,
        34, 122,  63, 118,  42,  84,  94,  81, 118, 110,  51, 123,  40,
        25,  60,  69,  31,  90,   0,  83,  11,  64,  24,  46,  77, 101,
        99,  73,  13,   2,  81,  25,  82,  17,   0,  90,  36,  26,  64,
        11, 127,   6,  84,  74, 113,  86,   7,  98,  70, 117,  60,   3,
       117,  15,  85,  91,  62,  57,  59,  43,  56,  13,  99,  70, 105,
        47,   0, 109,  73, 125, 102,  93,  15, 115,  59,  10,  44,  37,
        30,  26,  75, 127,  35,  62,  35, 112,  13])

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))

print("\nNext Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'Того раскаянье грызет.\n                        Все это часто придает\n                        Большую'

Next Char Predictions: 
 'rсьх1\'cэ9НДiжNчtуWОШЛулgшTCq{IФ\nН1uBbЖваВ3!ЛCМ7\nФPDu1ь)ОГоР,Я}тq"т5ПХsnpXm3а}жc\nкВъгЧ5рp0YQHDДьOsOн3'


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)

In [24]:
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 131)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.874953


In [25]:
model.compile(optimizer='adam', loss=loss)

In [26]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=100*5,
    save_weights_only=True,
    )

In [27]:
num_epoch = 40
history = model.fit(dataset, epochs=num_epoch, callbacks=[checkpoint_callback])

Epoch 1/40
44/44 [==============================] - 387s 9s/step - loss: 2.4698
Epoch 2/40
44/44 [==============================] - 388s 9s/step - loss: 2.2311
Epoch 3/40
44/44 [==============================] - 370s 8s/step - loss: 1.9328
Epoch 4/40
44/44 [==============================] - 391s 9s/step - loss: 1.7538
Epoch 5/40
44/44 [==============================] - 388s 9s/step - loss: 1.6703
Epoch 6/40
44/44 [==============================] - 391s 9s/step - loss: 1.5713
Epoch 7/40
44/44 [==============================] - 389s 9s/step - loss: 1.4959
Epoch 8/40
44/44 [==============================] - 388s 9s/step - loss: 1.4455
Epoch 9/40
44/44 [==============================] - 378s 9s/step - loss: 1.4123
Epoch 10/40
44/44 [==============================] - 379s 9s/step - loss: 1.3901
Epoch 11/40
44/44 [==============================] - 397s 9s/step - loss: 1.3669
Epoch 12/40
44/44 [==============================] - 392s 9s/step - loss: 1.3486
Epoch 13/40
44/44 [==================

In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_35'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            33536     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 131)            134275    
                                                                 
Total params: 30,592,899
Trainable params: 30,592,899


In [82]:
def generate_text(model, start_string):
  
    num_generate = 1000
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []

    temperature = 0.8

    model.reset_states()
    for i in range(num_generate):
        
        predictions = model(input_eval)
       
        predictions = tf.squeeze(predictions, 0)
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [83]:
text_ = generate_text(model, start_string=u"И вот идет уже ")

In [84]:
print(text_)

И вот идет уже еодуоооимеяояо.   таааоиииа ;, ,  


    


                                                         XXII

                                             Не в приетны послевает:
                                                         LV

                         Снег преизбают;
                              Тан? Продитла я ветелся,
                                            XVII

                                                XXXV

                                                            Татьяны модя подел,
                                Дужен вер                         LIII

                           Не приврестный долде. Вдольней скутой
                               Конь он он полного ильной,
                          И это под покревить, к там ро тобой
                                                                 В мы, что и ручи умачит
                                       Не полнал жену возбо                             Но можалы роченский други,
       

Как видно, без тщательной подгонки параметров генерации результат получается достаточно своеобразный